In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import importlib

# BOKEH
import bokeh.plotting as bk
import bokeh.models as bkmod
import bokeh.layouts as bklay

bk.output_notebook()

# Local imports
#------------------------------------------------
import WireDAQ.PandasPlus           # Make sure this import is after pandas
import WireDAQ.Constants as cst
import WireDAQ.NXCALS as nx
import WireDAQ.Parser as parser
import WireDAQ.Efficiency as eff

main = __import__('000_Efficiency_per_fill')


# Creating NXCALS variable containers
#------------------------------------------------
wires     = {'B1': [nx.NXCALSWire(loc = loc) for loc in ['L1B1','L5B1']],
             'B2': [nx.NXCALSWire(loc = loc) for loc in ['R1B2','R5B2']]}
beams     = [nx.NXCALSBeam(name) for name in ['B1','B2']]
LHC       = nx.NXCALSLHC()
b_slots   = np.arange(3564)
#------------------------------------------------


# Setting default values
#------------------------------------------------
_default_fig_width  = 2000
_default_fig_height = 400

_default_device = 'DBLM'

_default_import = 'local'
_default_path   = '/home/lumimod/work/run/data/2023/rawdata/'
_default_out    = '/eos/user/p/phbelang/www/Monitoring_BBCW/'
#------------------------------------------------




>>> Loading nx2pd.py version of 24.10.2022 @ 03:17PM



2023-05-25 13:23:19,418 [INFO] 
Limited Total Variation Regularization Support Detected! 
---> CVXPY is not installed. 
---> Many Total Variation Methods require CVXPY including: 
---> velocity, acceleration, jerk, jerk_sliding, smooth_acceleration
---> Please install CVXPY to use these methods.
---> Recommended to also install MOSEK and obtain a MOSEK license.
You can still use: total_variation_regularization.iterative_velocity

2023-05-25 13:23:19,421 [INFO] 
Limited Linear Model Support Detected! 
---> PYCHEBFUN is not installed. 
---> Install pychebfun to use chebfun derivatives (https://github.com/pychebfun/pychebfun/) 
You can still use other methods 

2023-05-25 13:23:19,421 [INFO] 
Limited Linear Model Support Detected! 
---> CVXPY is not installed. 
---> Install CVXPY to use lineardiff derivatives 
You can still use other methods 



---
# Loading data and run analysis
---

In [2]:
FILL        = 8773
import_from = 'local'

database,BCTF_efficiency,DBLM_efficiency,bb_df_b1,bb_df_b2 = main.run_analysis(FILL,import_from =import_from,local_path='/home/phbelang/002_raw_data/')
BOKEH_FIGS = {}

/home/phbelang/abp/WireDAQ/WireDAQ/Efficiency.py:176: RuntimeWarning: divide by zero encountered in divide
  eta      = cst.SIG_PP*data_L/(-dNdt)
/home/phbelang/abp/WireDAQ/WireDAQ/Efficiency.py:176: RuntimeWarning: divide by zero encountered in divide
  eta      = cst.SIG_PP*data_L/(-dNdt)
/home/phbelang/abp/WireDAQ/WireDAQ/Efficiency.py:288: RuntimeWarning: divide by zero encountered in divide
  lifetime = -data_N/(R_ellN)
/home/phbelang/abp/WireDAQ/WireDAQ/Efficiency.py:288: RuntimeWarning: invalid value encountered in divide
  lifetime = -data_N/(R_ellN)
/home/phbelang/abp/WireDAQ/WireDAQ/Efficiency.py:288: RuntimeWarning: divide by zero encountered in divide
  lifetime = -data_N/(R_ellN)
/home/phbelang/abp/WireDAQ/WireDAQ/Efficiency.py:288: RuntimeWarning: invalid value encountered in divide
  lifetime = -data_N/(R_ellN)


In [3]:
# Cropping the filling for testing
bb_df_b1 = bb_df_b1.loc[1070:2050]
bb_df_b2 = bb_df_b2.loc[1070:2050]

---
# Plotting Overview
---

In [11]:
# Reload main module
importlib.reload(main)
BOKEH_FIGS['Overview'] = main.make_overview_figure(FILL,database)

# Creating slider
ax,axis_name = main.new_axis(BOKEH_FIGS['Overview'],axis_name='slider')
time_slider  = main.Slider(BOKEH_FIGS['Overview'],x=5,y=5,w=0.5,h=20,y_range_name=axis_name)
time_slider.update(fill_color='black',fill_alpha=0.2,line_alpha=1,line_dash='2 2')

# Adding drawtool
slide_tool = PointDrawTool(renderers=[time_slider.renderer],icon=time_slider.icon)


bk.show(BOKEH_FIGS['Overview'])

AttributeError: unexpected attribute 'fill_color' to GlyphRenderer, possible attributes are coordinates, data_source, glyph, group, hover_glyph, js_event_callbacks, js_property_callbacks, level, muted, muted_glyph, name, nonselection_glyph, propagate_hover, selection_glyph, subscribed_events, syncable, tags, view, visible, x_range_name or y_range_name

In [ ]:





# Creating Figure
#=====================================
fig = bk.figure(height          = 500, 
                width           = 1500,
                title           = "Overview" + f' FILL {FILL:d}  ({database["Timestamp"].iloc[0].strftime("%Y-%m-%d")})', 
                x_axis_type     = "datetime",
                tools           = "pan,wheel_zoom,box_zoom,reset,save")
fig.xaxis.formatter= bkmod.DatetimeTickFormatter(hourmin = '%H:%M',hours='%H:%M',days='%H:%M',months='%H:%M',years='%H:%M')
fig.add_tools(bkmod.HoverTool(
    tooltips=[('Variable', '$name'),('Time (H:M)','$x{%H:%M}'),('Value','$y')],
    formatters={ "$x": "datetime"}))
#=====================================

# New axis function
#=====================================
def new_axis(fig,axis_name,side='left'):
    fig.extra_y_ranges[axis_name] = bkmod.Range1d(0,1)
    _ax = bkmod.LinearAxis(y_range_name=axis_name)
    fig.add_layout(_ax,side)

    return _ax,axis_name
#=====================================


# Plotting Intensity
#--------------------
for beam,color in zip(beams,['blue','red']):
    data = database.set_index('Timestamp')[beam.Intensity].dropna()
    
    fig.line(data.index,data,color=color,alpha=0.8,legend_label=beam.name,name=f'Intensity {beam.name}')

fig.yaxis.axis_label = "Intensity [1e14 p+]"
fig.xaxis.axis_label = f"Local Time, {database['Timestamp'].iloc[0].strftime('%Y-%m-%d')}"
#--------------------

# Plotting Luminosity
#--------------------
ax,axis_name = new_axis(fig,axis_name='Luminosity',side='left')
max_y = 0
for loc,color in zip(['ATLAS','CMS'],['orange','green']):
    data = database.set_index('Timestamp')[LHC['bb_Luminosity'][loc]].dropna()
    data = data.apply(lambda line: np.sum(line))
    
    fig.line(data.index,data,color=color,alpha=0.8,legend_label=loc,name=f'Luminosity {loc}',y_range_name=axis_name)

    max_y  = np.max((max_y,np.max(data)))

fig.extra_y_ranges[axis_name] = bkmod.Range1d(-0.05*max_y,1.05*max_y)
ax.axis_label = 'Luminosity [Hz/ub]'
#--------------------


# Plotting xing
#--------------------
ax,axis_name = new_axis(fig,axis_name='xing',side='right')


data = database.set_index('Timestamp')[LHC.Xing['IP5']].dropna()
fig.line(data.index,data,color='indianred',alpha=0.8,legend_label="theta/2",name="theta/2",y_range_name=axis_name)

    
fig.extra_y_ranges[axis_name] = bkmod.Range1d(110,180)
ax.axis_label = r"Half-crossing angle [urad]"
#--------------------


# Plotting beta star
#--------------------
ax,axis_name = new_axis(fig,axis_name='beta',side='right')


data = database.set_index('Timestamp')[LHC.betastar['IP5']].dropna()
data[(data>90)|(data<0)]=np.nan
fig.line(data.index,data,color='purple',alpha=0.8,legend_label="beta*",name="beta*",y_range_name=axis_name)

    
fig.extra_y_ranges[axis_name] = bkmod.Range1d(0,90)
ax.axis_label = r"Beta star [cm]"
#--------------------




# Plotting wire current
#--------------------
ax,axis_name = new_axis(fig,axis_name='wire',side='right')

for wire in wires['B2'] :
    data = database.set_index('Timestamp')[wire.I].dropna()

    fig.line(data.index,data,color='teal',alpha=0.8,legend_label="BBCW",name="BBCW",y_range_name=axis_name)

max_y = 2000
fig.extra_y_ranges[axis_name] = bkmod.Range1d(-0.05*max_y,1.05*max_y)
ax.axis_label = r"BBCW Current [A]"
#--------------------









# Legend Options
#=====================================
fig.legend.location     = "top_left"
fig.legend.click_policy = "hide"
#=====================================


# Display figure
#=====================================
bk.output_notebook()
bk.show(fig)
#=====================================

---
# Filling pattern
---

## TO DO:
```python
Add line showing average of highest efficiency (sort[-100:-1])
Add line showing average of lowest efficiency (sort[0:100])
````

In [ ]:


for beam,bb_df,color in zip(beams,[bb_df_b1,bb_df_b2],['royalblue','firebrick']):
    # Creating Figure
    #=====================================
    fig = bk.figure(height          = 200, 
                    width           = 1500,
                    title           = "Filled slots", 
                    tools           = "pan,wheel_zoom,box_zoom,reset,save",
                    active_drag     = "box_zoom")
    # fig.toolbar.active_drag = None
    # fig.toolbar.active_drag = "box_zoom"
    fig.grid.visible = False
    fig.add_tools(bkmod.HoverTool(
        tooltips=[('Variable', '$name'),('Bunch slot','$x{0}')]))
    #=====================================

    fig.vbar(x=bb_df.index, top=1, width=0.8,color=color,name=f"{beam.name}") 
    
    # Vertical line
    # l1 = bkmod.Span(location=0, dimension='height', line_color='black',line_alpha=0.5)
    # l2 = bkmod.Span(location=len(b_slots), dimension='height', line_color='black',line_alpha=0.5)
    # fig.renderers.extend([l1,l2])
    fig.xaxis.axis_label = "Bunch slot"
    fig.yaxis.axis_label = "Filled"
    fig.yaxis.ticker     = [0,1]
    fig.x_range          = bkmod.Range1d(-10, len(b_slots)+10)
    fig.y_range          = bkmod.Range1d(0, 1.05)
    

    
    BOKEH_FIGS[f'Filling {beam.name}'] = fig

# Linking axis
BOKEH_FIGS[f'Filling B2'].x_range = BOKEH_FIGS[f'Filling B1'].x_range
BOKEH_FIGS[f'Filling B2'].y_range = BOKEH_FIGS[f'Filling B1'].y_range

# Putting figs in a grid
BOKEH_FIGS['Fillings'] =  bklay.gridplot([[BOKEH_FIGS['Filling B1']], [BOKEH_FIGS['Filling B2']]])



# Display figure
#=====================================
bk.output_notebook()
bk.show(BOKEH_FIGS['Fillings'],sizing_mode='scale_both')
#=====================================


---
# Efficiency plot
---

### Compute data

In [ ]:
# BCTF
#==============================================================
# Computing efficiency 
BCTF_efficiency = {}
for beam in beams:
    BCTF_efficiency[beam.name]  = eff.compute_BCTF_efficiency(database,beam,dt= 60,smooth_derivative=False)
#==============================================================

# DBLM
#==============================================================
# Computing efficiency 
DBLM_efficiency = {}
for beam in beams:
    DBLM_efficiency[beam.name]  = eff.compute_dBLM_efficiency(database,beam,dt = 10,baseline = None,calib_ROI=None,filled_b_slots=None)
#==============================================================

---
# EXPORT HTML
---

In [ ]:
bk.output_file(filename=BOKEH_EXPORT, title="Bokeh Plot")
bk.save(fig)

---

In [ ]:

# CREATING FIGURE
#===============================
def Make_Fig(figsize= (25,8)):    
    fig, _axes = plt.subplots(figsize = figsize,ncols=1, nrows=2,gridspec_kw={'height_ratios': [1, 1]})
    plt.subplots_adjust(hspace=0.0)
    axes = {'B1'  :_axes[0],
            'B2'  :_axes[1]}
    
    return fig,axes


fig,axes = Make_Fig(figsize=(20,5))
for beam,bb_df,color in zip(beams,[bb_df_b1,bb_df_b2],['C0','C3']):

    plt.sca(axes[beam.name])
    plt.stem(bb_df.index,np.ones(len(bb_df)),basefmt='none',linefmt=color,markerfmt='none')
         


    plt.ylabel(f'Beam {beam.name[-1]}')

    plt.yticks([])
    plt.xlim([-50,np.max(buckets)+50])
    plt.ylim([0,1.1])

    plt.axvline(0,color='k',ls='-',alpha=0.5)
    plt.axvline(np.max(buckets),color='k',ls='-',alpha=0.5)


plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.05)

plt.sca(axes['B1'])
plt.xticks([])

plt.sca(axes['B2'])
plt.xlabel('Slot Number')
# plt.xlim([0,500])


In [ ]:
bb_df_b1

In [ ]:
bb_df_b1.groupby('Train Length').count()

---
# Efficiency
---

### From BCTF and DBLM

In [ ]:
# CREATING FIGURE
#===============================
def Make_Fig(figsize= (FIG_W,12)):

    fig = plt.figure(figsize=figsize)
    gs = mplGrid.GridSpec(2,1, wspace=0, hspace=0.05, height_ratios=[1,1], figure=fig)
    
    top  = fig.add_subplot(gs[0])
    bot  = fig.add_subplot(gs[1])


    for ax in [top]:
        ax.set_xticks([])

    axes = {'B1' :top,
            'B2' :bot}
    
    return fig,axes






fig,axes = Make_Fig(figsize=(FIG_W,8))

train = bb_df_b1.groupby('Train').get_group(1)
for beam in beams:




chosen_trains_list = [  [1,3,5],
                        [2,3,4,7,8,9,12,13,14]]
for df,datab,bb_df,strength,ax_ref,axlabel,axlabel2,chosen_trains in zip([MD_dblm,FILL8120_dblm],[database,database_FILL8120],[patt.b2.bb_schedule,FILL8120_patt.b2.bb_schedule],[MD_bb_strength,FILL8120_bb_strength],['left','right'],[f'MD8043',f'FILL 8120'],['Machine Development','Operational Fill'],chosen_trains_list):


    #===============================================================================
    # COLORCODED
    #===============================================================================
    
    sc_label = f'Effective number of BBLR'
    # BB_signature = np.nan*np.ones(len(buckets))
    # BB_signature[bb_df.index] =  bb_df['# of LR in ATLAS/CMS'].values
    # BB_signature = strength/np.max(strength) * np.max(bb_df['# of LR in ATLAS/CMS'].values)

    N_LR = bb_df.groupby('Train').get_group(chosen_trains[0])['# of LR in ATLAS/CMS'].values
    loc_max = np.argmax(N_LR)
    BB_signature = strength/strength[loc_max] * N_LR[loc_max]

    _colorby = BB_signature#[choosen_bunches]

    # if axlabel == 'MD8043':
    #     vmin = 15
    #     vmax = 45
    # else:
    #     vmin = 15
    #     vmax = 45
    vmin = 18
    vmax = 42
    cmap = 'cividis'
    #===============================
    
    # CHOOSE BUNCH
    # middles =  list(bb_df[bb_df['Tag'].isin(['22/48','23/48','24/48','25/48','26/48'])].index)
    # tagged = middles

    if axlabel == 'MD8043':
        tagged = bb_df[bb_df['# of LR in ATLAS/CMS']>10].index
    else:
        full_trains = np.tile([2,3,4],8) + np.repeat([0,5,10,15,20,25,30,35],3)

        tagged = bb_df[ (bb_df['# of LR in ATLAS/CMS']>10) & 
                        (bb_df['Tag'].apply(lambda x: float(x.split('/')[0])) <= 30) &
                        (bb_df['Tag'].apply(lambda x: float(x.split('/')[0])) > 3) & 
                        (bb_df.Train.isin(full_trains))].index

    

    



    plt.sca(axes[ax_ref])
    sc = plt.scatter(np.zeros(48),np.nan*np.zeros(48),c = _colorby,vmin=vmin,vmax=vmax,cmap=cmap)
    data   =  df[beam.name].set_index('Timestamp')['eta']
    for _b in tagged:
        color_idx = int(bb_df.loc[_b,'Tag'].split('/')[0])
        plt.plot(data.index,data.apply(lambda line: line[_b]),'-',alpha = 0.3,color=sc.to_rgba(_colorby[color_idx]))

    
    if ax_ref == 'right':
        cbar = plt.colorbar(fraction=0.05,pad=0.02)
        cbar.set_label(sc_label)

    # for ax,axlab in zip(axes,[f'{beam.name} BCTF',f'{beam.name} dBLM']):
    #     plt.sca(ax)
    #     plt.axhline(1,ls='--',color='k')
    #     plt.ylim([0.25,1.1])
    #     # plt.xlim([beta_start,MD_stop])
    #     plt.ylabel(r'$\sigma_{eff}$ (mb)',fontsize=15)

    #     show_wire_on(database,facecolor='k',alpha=0.2,label='Wires ON')
        

    #     # ADJUSTING LEGEND
    #     #====================================
    #     plt.legend()
    #     # handles, labels = plt.gca().get_legend_handles_labels()
    #     line0, = plt.plot([np.nan],[np.nan],'-',alpha=0.5,color=sc.to_rgba(vmax))
    #     line1, = plt.plot([np.nan],[np.nan],'-',alpha=0.5,color=sc.to_rgba(np.mean([vmin,vmax])))
    #     line2, = plt.plot([np.nan],[np.nan],'-',alpha=0.5,color=sc.to_rgba(vmin))

    #     MDplt.add_multi_V_legend([line0,line1, line2],axlab)

    #     #====================================

    plt.axhline(1,ls='--',color='k')
    plt.ylim([0.2,1.09])
    # show_wire_on(df,facecolor='k',alpha=0.2,label='Wires ON')

    if axlabel == 'MD8043':
        plt.xlim([ pd.Timestamp('2022-11-05 21:00:00',tz=TZONE), pd.Timestamp('2022-11-06 01:00:00',tz=TZONE)])
    else:
        plt.xlim([ pd.Timestamp('2022-08-12 21:15:00',tz=TZONE), pd.Timestamp('2022-08-12 23:15:00',tz=TZONE)])
    
    plt.xticks(list(plt.gca().get_xticks())[1:-1])
    
    if axlabel != 'MD8043':
        # get every 30 min
        plt.xticks(list(plt.gca().get_xticks())[::2])
    else:
        plt.yticks([0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

    plt.xlabel(f'Time   [{datab["Timestamp"].iloc[0].strftime("%Y-%m-%d")}]')
    

     # Adding axlabel
    props = dict(boxstyle='round', facecolor='white', alpha=0.95)
    if axlabel == 'MD8043':
        x_text,y_text = 0.03/2,0.97
    else:
        x_text,y_text = 0.03,0.97
    plt.text(x_text,y_text, axlabel2, fontsize=14, 
                                    transform=plt.gca().transAxes,
                                    va='top',ha='left', bbox=props)


    show_wire_on(datab,facecolor='C2',alpha=0.3,zorder=-100,label='Wires ON')

    # xing
    ax_xing = plt.gca().twinx()
    xing_color = 'indianred'
    datab.nxPlot('Timestamp',LHC.Xing['IP5'],lw=2,color=xing_color,alpha=0.8)


        
   

    
    plt.ylim([110+15,180+15])

    if axlabel != 'MD8043':
        ax_xing.set_yticks([])
    else:

        # axes['left'].spines["left"].set_position(("axes", -0.1))

        ax_xing.yaxis.set_ticks_position("left")
        ax_xing.yaxis.set_label_position("left")
        ax_xing.spines["left"].set_position(("axes", -0.1))
        ax_xing.set_frame_on(True)
        ax_xing.patch.set_visible(False)
        ax_xing.set_yticks([130,140,150,160])
        ax_xing.set_ylabel(r'Half-crossing angle, $\theta_c/2$ ($\mu$rad)')
        ax_xing.tick_params(axis='y', colors=xing_color)
        ax_xing.spines['left'].set_color(xing_color)
        axes['left'].set_ylabel(r'Bunch-by-bunch efficiency, $\eta$')


plt.sca(axes['left'])
plt.plot(np.nan,np.nan,color=xing_color,alpha=0.8,label=r'Half-crossing angle, $\theta_c/2$')
# ADJUSTING LEGEND
#====================================
plt.legend()
# handles, labels = plt.gca().get_legend_handles_labels()
line0, = plt.plot([np.nan],[np.nan],'-',alpha=0.5,color=sc.to_rgba(vmax))
line1, = plt.plot([np.nan],[np.nan],'-',alpha=0.5,color=sc.to_rgba(np.mean([vmin,vmax])))
line2, = plt.plot([np.nan],[np.nan],'-',alpha=0.5,color=sc.to_rgba(vmin))

MDplt.add_multi_V_legend([line0,line1, line2],r'Bunch-by-bunch efficiency, $\eta$',framealpha=0.95)

#====================================


plt.tight_layout()

In [ ]:
import numpy as np

from bokeh.layouts import column, grid
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, output_file, show, output_notebook

# output_file('dashboard.html')

tools = 'pan'


def bollinger():
    # Define Bollinger Bands.
    upperband = np.random.randint(100, 150+1, size=100)
    lowerband = upperband - 100
    x_data = np.arange(1, 101)

    # Bollinger shading glyph:
    band_x = np.append(x_data, x_data[::-1])
    band_y = np.append(lowerband, upperband[::-1])

    p = figure(x_axis_type='datetime', tools=tools)
    p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

    p.title.text = 'Bollinger Bands'
    p.title_location = 'left'
    p.title.align = 'left'
    p.width = 800
    p.height = 600
    p.grid.grid_line_alpha = 0.4
    return [p]


def slider():
    x = np.linspace(0, 10, 100)
    y = np.sin(x)

    source = ColumnDataSource(data=dict(x=x, y=y))

    plot = figure(
        y_range=(-10, 10), tools='', toolbar_location=None,
        title="Sliders example")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
    freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
    phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase")
    offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset")

    callback = CustomJS(args=dict(source=source, amp=amp_slider, freq=freq_slider, phase=phase_slider, offset=offset_slider),
                        code="""
        const data = source.data;
        const A = amp.value;
        const k = freq.value;
        const phi = phase.value;
        const B = offset.value;
        const x = data['x']
        const y = data['y']
        for (let i = 0; i < x.length; i++) {
            y[i] = B + A*Math.sin(k*x[i]+phi);
        }
        source.change.emit();
    """)

    amp_slider.js_on_change('value', callback)
    freq_slider.js_on_change('value', callback)
    phase_slider.js_on_change('value', callback)
    offset_slider.js_on_change('value', callback)

    widgets = column(amp_slider, freq_slider, phase_slider, offset_slider)
    return [widgets, plot]


def linked_panning():
    N = 100
    x = np.linspace(0, 4 * np.pi, N)
    y1 = np.sin(x)
    y2 = np.cos(x)
    y3 = np.sin(x) + np.cos(x)

    s1 = figure(tools=tools)
    s1.circle(x, y1, color="navy", size=8, alpha=0.5)
    s2 = figure(tools=tools, x_range=s1.x_range, y_range=s1.y_range)
    s2.circle(x, y2, color="firebrick", size=8, alpha=0.5)
    s3 = figure(tools='pan, box_select', x_range=s1.x_range)
    s3.circle(x, y3, color="olive", size=8, alpha=0.5)
    return [s1, s2, s3]

l = grid([
    bollinger(),
    slider(),
    linked_panning(),
], sizing_mode='stretch_both')

output_notebook()
show(l)